# 非凸优化：BGD的opt method

凸函数可以收敛到唯一最优解，非凸函数则只能收敛到local min，所以非凸函数的收敛目标是：收敛快，收敛到一个相对好的local min。

<font color=norange>**model框架：**
$$
input \rightarrow \underset{optimization}{{\underbrace{\overset{f(x,w)}{\overbrace{feature\ representation + classifier}} \rightarrow \overset{Div,reg}{ \overbrace{ cost function}}}}} 
$$</font>

**整个model可以拆分为两大类问题：模型结构设计和优化求解**
1. 设计一套好的模型结构，包括选择三件套$f(x,w), Div和Reg$，使min expected Loss能尽可能接近min Error这一真实目标
2. 选择好的优化方法(optimization method)，能以高效的方式找到相对好的local min

**梯度下降是求解优化问题的标准方法，但实践中有两类典型问题要解决**：
1. loss surface形态不好带来的各种问题
2. 网络深度带来的计算问题

## I. 第一类问题：loss surface形态不好

### I.1 凸函数condition number大可能引起oscillate和收敛慢
<font color=green>**solution：Adam = Momentum + RMSprop, input normalization**</font>
1. **oscillate** \
(1) 发生原因：单一learning rate时，为了让收敛速度不至于太慢，有的维度$η_{i, opt}<η_{i}<2*η_{i, opt}$，此时，这些维度上梯度符号会正负交互，converge trajectory会发生oscillate。\
<font color=orange>注：和fluctuate不同，oscillate的方向是对的，fluctuate方向是错的。fluctuate是SGD因为抽样的随机性使minibatch得到的梯度$\ne$偏离真实梯度方向而发生。</font> \
(2) solution: 用梯度的历史平均来决定方向，平滑掉波动，如：**Momentum，Nesterov** 
2. 收敛慢：\
(1) 发生原因：当$\eta$时scalar时，为迁就$\lambda$较大的方向而调低了$\eta$大小，使得$\lambda$较小的那些方向上的收敛速度变慢。\
(2) solution：不同的维度用不同的learning rate，adaptive learning rate。如：**RMSprop**

### I.2 非凸函数: random restart, SGD + lr decay, skip connection
1. saddle point：\
(1) 描述：在该点处gradient=0，迭代停止，但hessian的特征值不是都大于0，而是有正有负。\
(2) solution: Momentum正好解决了这个问题
2. inflection point:\
(1) 描述：在该点处gradient=0，，迭代停止，但hessian的特征值也是0。在图形上是连续的“一字型平地”。\
(2) solution: Momentum一定程度上解决了这个问题
3. valley：\
(1) 描述：水流冲开山谷形成的深V形的峡谷。这种形态容易发生oscillate。\
(2) solution: adaptive learning rate
4. plateau: \
(1) 描述：A plateau is a flat-topped elevated landform从周围平缓地势上拔地而起. \
(2) solution: RMSprop可以减轻这个问题
5. poor local min \
(1) 描述：指该local min与global min差距很大 \
(2) solution：**random restart，SGD+learning rate decay**的效果更好，因为SGD寻址的范围更广
 - **learning rate decay**: 需要的假设是，开口大的位置local min更好，开口小的位置local min不好，所以一开始用大的learning rate可以escape from这些形态不好的位置，直到找到还不错的位置（error rate有了大幅下滑后），再逐步降低learning rate，在选中区域的local min位置converge
6. nice local min(指该local min与global min接近)附近的loss surface形态不好 \
(1) 描述：比如high gradient near the min，low gradient far from the min. \
(2) solution：选择好的divergence function和model 结构。如：skip connection[参考：percy liang对loss land scape的研究]

## II. 第二类问题：‘深度’引发的计算问题
### II.1 forward propagation中的问题
#### II.1.1 Forward Propagation
$$
\begin{align}
z^{(1)}&=f(s^{(0)})=f(z^{(0)}@w^{(0)}) \\
z^{(2)}&=f(s^{(1)})=f(z^{(1)}@w^{(1)})=f(f(w^{(0)})@w^{(1)})\\
z^{(3)}&=f(s^{(2)})=f(z^{(2)}@w^{(2)}) \\
&=f(f(z^{(1)}@w^{(1)})@w^{(2)})=f(f(f(z^{(0)}@w^{(0)})@w^{(1)})@w^{(2)})\\
...\\
z^{(h)}&=f(s^{(h-1)})=f(z^{(h-1)}@w^{(h-1)})\\
&=f\left(...f\left (f(f(z^{(0)}@w^{(0)})@w^{(1)})@w^{(2)}\right )...@w^{(h-1)}\right ) \\
...
\end{align}
$$
 - 最后一步$s^{(H)}=w^{(H)}@z^{(H)}$得到score s的形状是$(N, D_{out})$,classification问题中，通常后续运算是先将score matrix转化成长度为N的vector，在numpy中形状是$(N,)$，最后再转变成loss scalar。

2. 参数的dimension变化
$$
\begin{align}
&\ \  z^{(0)}(N,D_0),\ \ z^{(1)}(N, D_1),\ \  ... ,z^{(h-1)}(N, D_{h-1}),z^{(h)}(N, D_h),..., z^{(H)}(N, D_{h}) \\
& w^{(0)}(D_0, D_1), w^{(1)}(D_1, D_2), ..., w^{(h-1)}(D_{h-1}, D_h),\ \ \  ...\ \ \  ...,\ \ \  w^{(H)}(D_h, D_{out})
\end{align}
$$

#### II.2.2 FP中的问题：saturated neurons
1. 问题说明：由activation function运算特征和weight初始化取值带来的问题
 - **以tanh、sigmoid为例：** \
   (1)**如果$w^{(i)}$中元素scale(eg:体现在variance)小**，eg: $|w^{(i)}|\approx β < 0.001$。层度加深后，从某一层开始，后面各层$f(z)$→0 \
   (2)**如果$w^{(i)}$中元素scale大**，eg: $|w^{(i)}|\approx β > 0.1$。层度加深后，从某一层开始，后面各层$f(z)$→1 \
   上面两种情况下，前向信息传递都失效，$Var(f(z))\approx0$。还造成梯度消失。

#### II.2.3 FP中的问题：covariate shift     
1. 理想状态
- **假设1**：$w_i$的值给定。那么只要最初的输入$z_0=x$的分布不变，则线性变化$w_0·x$的分布也稳定，作为下一层输入的$z_1=f(w_0·x)$的分布也稳定。同理，后续每一层的分布也稳定。
- **放松为假设2**：$w_i$的值在每次迭代时都发生变化，但是能保证线性变化$w_0·x$的分布稳定。此时，作为下一层输入的$z_1=f(w_0·x)$的分布也稳定。同理，后续每一层的分布也稳定。

2. **实际问题**：\
(1) 每个batch得到的数据与其他batch数据的分布都很难保证一致。假设$w$不变，每层拿到的数据都因为最初的x batch变化而变化。\
(2) 更严重的是，迭代会导致$w$的值持续变化，加大$w_0·x$的分布变化，后续每一层的输入$z_{i+1}=f(w_{i}·z_{i})$分布也都变化。
- 简化以单层网络为例来说明，如果input的分布总是变化，也就相当于每次输入对应的loss surface都在变化，$w$很难收敛。这比mini-batch SGD的情况还糟糕，因为mini-batch中的分布是一样的，只是因为抽样导致了样本特征变化。
3. **solution:batch norm** \
batchnorm给出的处理方式是锁定input的分布。相当于一种Regularization，各层w的取值保持各自分布特征(均值和方差)不变。

### II.2 backward propagation中的问题

#### II.2.1 Backward propagation
 - 反向求梯度时，$ds^{(H)}的shape是(N, D_{out})$
 - NN的最后一层H层的梯度是：$$\begin{align}
dw^{(H)}&=\left(z^{(H)}\right)^T@ds^{(H)}=\left(z^{(H)}\right)^T@dscore\\
dz^{(H)}& =ds^{(H)}@\left(w^{(H)}\right)^T=dscore@\left(w^{(H)}\right)^T \end{align}$$
 - 有：
$$
\begin{align}
ds^{(h)} & = dz^{(h+1)}*\nabla f^{(h)} \\
& = ds^{(h+1)}@\left(w^{(h+1)}\right)^T*\nabla f^{(h)}\\
& = ds^{(H)}@\left(w^{(H)}\right)^T*\nabla f^{(H-1)}...@\left(w^{(h+2)}\right)^T*\nabla f^{(h+1)}@\left(w^{(h+1)}\right)^T*\nabla f^{(h)}\\
& = \underset{N,D_H\times D_H,D_{H-1}}{\underbrace{dscore@\left(w^{(H)}\right)^T}}  *\underset{N,D_{H-1}}{\underbrace{\nabla f^{(H-1)}}}  ...@\underset{D_{h+2},D_{h+1}}{\underbrace{\left(w^{(h+2)}\right)^T}}  *\underset{N,D_{h+1}}{\underbrace{\nabla f^{(h+1)}}}  @\underset{D_{h+1},D_{h}}{\underbrace{\left(w^{(h+1)}\right)^T}}  *\underset{N,D_{h}}{\underbrace{\nabla f^{(h)}}}
\end{align}
$$
 - 代入前面的式子有：
 $$
\begin{align}
dw^{(h)}&=\left(z^{(h)}\right)^T@ds^{(h)} \\
&=\left(z^{(h)}\right)^T@\color{Red}{dscore@\left(w^{(H)}\right)^T*\nabla f^{(H-1)}...@\left(w^{(h+2)}\right)^T*\nabla f^{(h+1)}@\left(w^{(h+1)}\right)^T*\nabla f^{(h)}}
\end{align} 
 $$

#### II.3.1 梯度消失
1. 说明：ReLU, sigmoid, tanh都有$0< \nabla f_{z}<1$的特点，随着网络深度增加，$\frac{\partial Loss}{\partial x_{i}}$中元素越来越多趋于0。


#### II.3.2 梯度爆炸
 - <font color=norange>在RNN中，$w^{(i)}=w^{(i)}=w$，此时梯度公式为：
$$
ds^{(h)} = dscore@w^T*\nabla f^{(H-1)}@w^T*\nabla f^{(H-2)}...@w^T*\nabla f^{(h+1)}@w^T*\nabla f^{(h)} 
$$</font>

1. 说明：根据矩阵乘法的特点，$wx$在$w$的singular value小于1的方向上shrink x，此时梯度消失情况会被加重。在$w$的singular value大于1的方向上expand x，此时可能导致gradient explode。

2. **sigmoid发生梯度消失：**
 - $f(z^{(i)})=0$或=1时，$\bigtriangledown f_{z^{(i)}}=f(z^{(i)})*(1-f(z^{(i)}))=0$。只要最后一层为0，整个网络的$w$的梯度都为0
 - 这里是按元素归零：只要$z_i$中有元素为0，那么$\bigtriangledown f_{z_i}$在对应位置取0。
 - *公式(6)可知，只要某层梯度**整体→0**，比它浅层的所有梯度都→0。*
 - 此外，$f(z^{(i)})=0$时，$z^{(i+1)}=w·f(z^{(i)})=0$，公式(7)可知$w^{(i)}$梯度为0。此时，$i$层之后的所有层按照运算特征也都会有$f(z^{(j)})=0$。所以，$i$层之后的各层$w^{(j)}$梯度都为0.
 - 但这里要求$f(z_i)$中元素***整体→0***。

3. **tanh发生梯度消失：**
 - $|f(z^{(i)})|=1$时，$\bigtriangledown f_{z^{(i)}}=1-f^{2}(z^{(i)})=0$。只要最后一层为0，整个网络的$w$的梯度都为0
 - *公式(6)可知，只要某层梯度**整体→0**，比它浅层的所有梯度都→0。*
 - $f(z^{(i)})=0$时，$z^{(i+1)}=w·f(z^{(i)})=0$，公式(7)可知$w^{(i)}$梯度为0。此时，$i$层之后的所有层按照运算特征也都会有$f(z^{(j)})=0$。所以，$i$层之后的各层$w^{(j)}$梯度都为0。**和sigmod中不同，此时比$i$层更浅的各层此时还能继续迭代。**
 - 但这里也要求$f(z^{(i)})$中元素***整体→0***。

**理解该问题的3个初步视角**：
1. 选择性质更合适的activation：tanh和sigmoid都有这个问题，tanh没有sigmoid严重，所以RNN用tanh，不用sigmoid。CNN用Relu
2. 让f(z)不趋于相同值，即$Var(f(z))≠0$：Xaiver，MSRA(Kaiming)
   - 通过weight initialization可以在刚开始迭代的时候近似满足$Var(z_{i+1}))=Var(z_{i})$，缓解上述问题。ResNet中是$Var(f(w_{i}x_{i})+x)=Var(x_{i})$
   - 但一旦开始迭代后，$w$的scale取决于learning rate和梯度的大小，仅靠初始值的上述安排无法保证迭代过程中$Var(z_{i+1}))=Var(z_{i})$继续成立。
3. 在初始化的基础上，为了控制迭代过程中$w$的scale在合理区间，不变得太大，发生saturate，只能用small learning rates，代价是降低了converge rate。
   - 但总得来说，**以上方法都没有从本质上解决问题。**

**一种更强的约束是Batchnorm**
 - 让每一层的input的分布保持稳定$distribution(z^{[t+1]}_i)=distribution(z^{[t]}_i)$，而不仅是让每一层input的方差保持层间的一致性$Var(z_{i+1}))=Var(z_{i})$。前者和initialization方法一样可以让$Var(z)≠0$，在此基础上，附加了更多约束。并且还有一个优点是，initialization的各种方法难以在迭代过程中持续保证$Var(z_{i+1}))=Var(z_{i})$。而Batchnorm则可以做到。
 - 图示：MLP中不同initial weight scale下，batchnorm效果
<img src="pics/batchnormvsscale.png" alt="alt text" width="420"/>

#### II.2.3 如果activation outputs总大于0，会导致SGD时w的迭代呈现zigzag形态，迭代速度慢。
1. 问题说明：单样本SGD时，由于每次只有一个样本，此时$w$中同一列元素迭代时，由梯度决定的迭代方向限制在全+和全-空间，发生zigzag，降低迭代速度。<font color=red>sigmoid和relu有这个问题。</font>
 - 分析：
$$
\begin{align}
ds^{(h)} 
& = \nabla f^{(h)} * dz^{(h+1)} \\
dw^{(h)}
&=\left(z^{(h)}\right)^T@ds^{(h)} =\left(f^{(h-1)}\right)^T@\nabla f^{(h)} * dz^{(h+1)} \\
简记为：\\
dw & = z^T@\nabla f*dz^{'}\\
& = \left(z^T\right )_{D\times 1}@\begin{pmatrix}
\nabla f_1dz^{'}_1 & \nabla f_2dz^{'}_2 & ... & \nabla f_{D^{'}}dz^{'}_{D^{'}}
\end{pmatrix}_{1\times D^{'}} \\
dw中的第j列：\\
dw_{·j}& = \nabla f_jdz^{'}_j * z^T，其中，\nabla f_j>0,z^T=(f^{(h-1)})^T>0
\end{align} 
$$
**结论：<font color=norange>dw的shape是$(D,D^{'})$，其中每一列的符号由其对应的scalar：$dz_j^{'}$决定，每次迭代的方向都只能同+或同-。</font>**

2. <font color=green>mini-batch和Batch GD这个问题不大。因为多个样本有正有负互相抵消。</font>
 - 分析：假设每个batch有b个样本，和上面分析类似，有：
 $$
\begin{align}
dw & = z^T@\nabla f*dz^{'}\\
& = \left(z^T\right )_{D\times b}@\begin{pmatrix}
\nabla f_1dz^{'}_1 & \nabla f_2dz^{'}_2 & ... & \nabla f_{D^{'}}dz^{'}_{D^{'}}
\end{pmatrix}_{b\times D^{'}} \\
dw中的第j列：\\
dw_{·j}& = \left(z^T\right )_{D\times b}@\left (\nabla f_jdz^{'}_j\right )_{b\times 1}
\end{align}
 $$
 此时，$dz_j^{'}$是vector，它包括的元素有正有负，所以dw中每一列的元素也有正有负

3. <font color=green>tanh没有这个问题，因为tanh函数的值有正有负</font>
4. 把上面的分析一般化，不仅是全正全负的activation out不利于求解，均值不为0都有类似问题。 \
<font color=red>**这也给preprocess data提供了一个zero-center操作的理由。因为x作为输入就是这里的$z_0$**</font>